In [1]:
from finetune_class import FineTunedModel

2024-04-07 12:04:10.020813: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 12:04:10.046808: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 12:04:10.831090: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model = FineTunedModel()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
model("Is there an engineering major at SUNY Brockport?", temperature=0.5)

"Yes, indeed! SUNY Brockport offers an exciting Bachelor of Science in Engineering program with several concentrations such as Computer Engineering, Electrical Engineering, Mechanical Engineering, and Materials Science & Engineering. It's a fantastic way to gain a solid foundation in science and mathematics while exploring the fascinating world of engineering."

In [5]:
questions = [
    "Is there a math major at SUNY Brockport?",
    "Is there an engineering major at SUNY Brockport?",
    "Tell me about the nursing major.",
    "How can I get involved in undergraduate research?",
    "How can I get involved in clubs?",
    "What options are there for financial aid at Brockport?",
    "How can I report sexual misconduct?",
    "Can I get credit for completing an internship in the math department?",
    "Is there a gym available to students on campus?",
    "How can I apply to SUNY Brockport?",
    "What is the meaning of life?",
]

with open("output.txt", "w") as f:
    for q in questions:
        f.write(f"Question: {q}\n")
        f.write(f"Answer: {model(q, temperature=0.01)}\n")
        f.write("\n")

/home/msaad/miniconda3/envs/thesis/lib/python3.11/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [7]:
import pandas as pd

df = pd.DataFrame(
    {
        "Question": questions,
        "Answer": [model(q, temperature=0.01) for q in questions],
    }
)

df.to_csv("output.csv", index=False)

In [9]:
df['Question'].to_list()

0              Is there a math major at SUNY Brockport?
1      Is there an engineering major at SUNY Brockport?
2                      Tell me about the nursing major.
3     How can I get involved in undergraduate research?
4                      How can I get involved in clubs?
5     What options are there for financial aid at Br...
6                   How can I report sexual misconduct?
7     Can I get credit for completing an internship ...
8       Is there a gym available to students on campus?
9                    How can I apply to SUNY Brockport?
10                         What is the meaning of life?
Name: Question, dtype: object

In [1]:
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    TextIteratorStreamer, 
    BitsAndBytesConfig, 
    pipeline,
    GenerationConfig
)
from typing import Optional
from threading import Thread
import torch

tokenizer=AutoTokenizer.from_pretrained("msaad02/BrockportGPT-7b", device_map={"": 0})
model=AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="msaad02/BrockportGPT-7b",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    ),
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)

2024-04-07 11:42:27.455868: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-07 11:42:27.639693: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-07 11:42:28.449103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
prompt = lambda question: f"""\
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant for SUNY Brockport, a public college in Brockport, New York. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

{question} [/INST]
"""

In [147]:
question = prompt("Is there a math major at SUNY Brockport?")

inputs = tokenizer(question, return_tensors="pt").to("cuda")

In [148]:
tokens = [tokenizer.decode(i) for i in range(32000)]

In [152]:
output = model.generate(
    **inputs, 
    generation_config=GenerationConfig(
        output_scores=True,
        do_sample=True,
        max_new_tokens=15,
        temperature=0.01
    ),
    return_dict_in_generate=True
)

In [153]:
# temperature = 0.01
probs = torch.nn.functional.softmax(output['scores'][0][0], dim=0)
argsort = probs.argsort(descending=True)
d = {token: round(prob*100,3) for token, prob in zip([tokens[idx] for idx in argsort[0:10]], probs[argsort][0:10].cpu().numpy())}
d

{'Yes': 100.0,
 'Ab': 0.0,
 '<unk>': 0.0,
 '<s>': 0.0,
 '</s>': 0.0,
 '\x00': 0.0,
 '\x01': 0.0,
 '\x02': 0.0,
 '\x03': 0.0,
 '\x04': 0.0}

In [150]:
# temperature = 1.0
probs = torch.nn.functional.softmax(output['scores'][0][0], dim=0)
argsort = probs.argsort(descending=True)
d = {token: round(prob*100,3) for token, prob in zip([tokens[idx] for idx in argsort[0:10]], probs[argsort][0:10].cpu().numpy())}
d

{'Yes': 29.019,
 'Ab': 22.6,
 'No': 7.81,
 'Current': 5.368,
 'Wh': 4.737,
 'S': 4.45,
 'At': 3.466,
 'We': 2.699,
 'There': 2.536,
 'Un': 2.382}

In [82]:
probs = torch.nn.functional.softmax(output['scores'][1][0], dim=0)

In [94]:
probs.cpu().numpy().round(2).argmax()

5890

In [95]:
probs.cpu().numpy().round(2)[5890]

0.99

In [80]:
probs['lying']

tensor(0., device='cuda:0')

In [68]:
probsprobs = torch.nn.functional.softmax(output['scores'][1][0], dim=0)
a = probsprobs[0].cpu()

tensor(0.)

In [52]:
probsprobs.argmax()

tensor(5890, device='cuda:0')

{'<unk>': 0.0,
 '<s>': 0.0,
 '</s>': 0.0,
 '\x00': 0.0,
 '\x01': 0.0,
 '\x02': 0.0,
 '\x03': 0.0,
 '\x04': 0.0,
 '\x05': 0.0,
 '\x06': 0.0,
 '\x07': 0.0,
 '\x08': 0.0,
 '\t': 0.0,
 '\n': 0.0,
 '\x0b': 0.0,
 '\x0c': 0.0,
 '\r': 0.0,
 '\x0e': 0.0,
 '\x0f': 0.0,
 '\x10': 0.0,
 '\x11': 0.0,
 '\x12': 0.0,
 '\x13': 0.0,
 '\x14': 0.0,
 '\x15': 0.0,
 '\x16': 0.0,
 '\x17': 0.0,
 '\x18': 0.0,
 '\x19': 0.0,
 '\x1a': 0.0,
 '\x1b': 0.0,
 '\x1c': 0.0,
 '\x1d': 0.0,
 '\x1e': 0.0,
 '\x1f': 0.0,
 '': 0.0,
 '!': 0.0,
 '"': 0.0,
 '#': 0.0,
 '$': 0.0,
 '%': 0.0,
 '&': 0.0,
 "'": 0.0,
 '(': 0.0,
 ')': 0.0,
 '*': 0.0,
 '+': 0.0,
 ',': 0.0,
 '-': 0.0,
 '.': 0.0,
 '/': 0.0,
 '0': 0.0,
 '1': 0.0,
 '2': 0.0,
 '3': 0.0,
 '4': 0.0,
 '5': 0.0,
 '6': 0.0,
 '7': 0.0,
 '8': 0.0,
 '9': 0.0,
 ':': 0.0,
 ';': 0.0,
 '<': 0.0,
 '=': 0.0,
 '>': 0.0,
 '?': 0.0,
 '@': 0.0,
 'A': 0.0,
 'B': 0.0,
 'C': 0.0,
 'D': 0.0,
 'E': 0.0,
 'F': 0.0,
 'G': 0.0,
 'H': 0.0,
 'I': 0.0,
 'J': 0.0,
 'K': 0.0,
 'L': 0.0,
 'M': 0.0,
 'N': 0.0,


In [57]:
round(float(probsprobs[5890].cpu()),3)

0.989

In [54]:
probsprobs[5890]

tensor(True, device='cuda:0')

In [46]:
probs['lying']

0.0

In [45]:
{token: prob for token, prob in probs.items() if prob > 0.01}

{}

In [30]:
tokenizer.decode(5890)

'lying'

In [31]:
print(tokenizer.decode(output['sequences'][0]).split("[/INST]\n")[1])

Applying to SUNY Brockport is easy! You can start
